# Preparation Script

### Setup

In [4]:
!pip install -r requirements.txt

In [24]:
import wiki_dump_parser
import sys

from os import listdir, remove, system, devnull
from os.path import join
import urllib.request
import pathlib
import subprocess


# Disable
def blockPrint():
    sys.stdout = open(devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

dataFolder="./data/"
backupDate="20200401"
language = 'en' # We can also loop through different languages as next step.

### Download Files
First the files will be downloaded from given target.

Then we need to convert xml dumps to csvs for further processing. The files should be residing in ```data/raw/xml```. Don't forget putting xml's under their respective folder, ex: ```data/raw/xml/en/enwiki-20200401-stub-meta-history1.xml```.

In [25]:
def processXmlData(xmlDataFile):
    print(f"Processing {xmlDataFile}")
    blockPrint()
    wiki_dump_parser.xml_to_csv(xmlDataFile)
    enablePrint()
    csvFileName = xmlDataFile.replace(".xml",".csv")
    remove(xmlFileName)

    # Convert the csv to parquet for faster processing.
    csv_data = pd.read_csv(csvFileName, quotechar ='|')
    parquetFileName = csvFileName.replace(".csv", ".parquet")
    csv_data.to_parquet(parquetFileName, engine='pyarrow')
    remove(csvFileName)
    print(f"Completed Processing {xmlDataFile}!")


def processZip(zipPath):
    print(f"Extracting: {zipPath}")
    result = subprocess.run(['unpigz', '-d', zipPath], capture_output=True)
    extractedFileName = zipPath.replace(".gz","")
    print(f"Extracted file: {zipPath}")
    processXmlData(extractedFileName)


packs = list(map(lambda x: x+1, range(27)))

for currentPack in packs:
    languageFolder = f"{dataFolder}{language}"
    pathlib.Path(languageFolder).mkdir(parents=True, exist_ok=True)
    
    fileName = f"enwiki-{backupDate}-stub-meta-history{currentPack}.xml.gz"
    backupUrl = f"http://ftp.acc.umu.se/mirror/wikimedia.org/dumps/{language}wiki/{backupDate}/{fileName}"

    filePath = f"{languageFolder}/{fileName}"
    urllib.request.urlretrieve(backupUrl, filePath)
    
    processZip(filePath)
    
    

NameError: name 'xmlFileName' is not defined